<div class="alert alert-block alert-info" style="background-color: white; text-align: center;">
  <h1> BUSINESS CASE 3: Recommendation System for Recheio </h1>
  <h2> Group E _ PART 2 </h2>
    
  <p>  </p>
    
  <p> Business Cases for Data Science </p>
  <p> Masters in Data Science Spealization in Business Analytics - 2023/2024 </p>
    
  <p>  </p>
    
  <p><strong style="font-weight: bold;"> Group Members: </strong></p>
  <p> - Duarte Mendes 20230494 <p>
  <p> - Dzmitry Nisht 20230776 <p>
  <p> - Inês Silva 20201580 <p>
  <p> - José Marcal 20201581 <p>
  <p> - Ricardo Sousa 20201611 <p>
    
</div>

### **Table of Contents**

* [1. Notebook Setup](#1)
* [2. Modelling](#2)
    * [2.1. Content-Based Filtering](#2_1)
    * [2.2. Collaborative Filtering](#2_2)
    * [2.3. Popularity-Based Recommendations](#2_3)
        * [2.3.1. General Popularity](#2_3_1)
        * [2.3.2. Popularity per Type of Client](#2_3_2)
        * [2.3.3. Popularity per Client](#2_3_3)
        * [2.3.4. "Did you forget?" approach](#2_3_4)
    * [2.4 Smart Basket Recommendation](#2_4)
    * [2.5 Page Rank Recommendation](#2_5)
* [3. Results](#3)

<hr>

<a class="anchor" id="1">

# 1. Notebook Setup: Packages and Module Loading

</a>

In [1]:
# basic imports
import pandas as pd # 1.5.3
import numpy as np # 1.26.4
import seaborn as sns # 0.12.2
import matplotlib.pyplot as plt # 3.7.1
from datetime import datetime, timedelta
import time

# Modelling
from sklearn.model_selection import train_test_split # 1.3.2
from sklearn.metrics.pairwise import cosine_similarity # 1.3.2
from sklearn.metrics import pairwise_distances #1.3.2
import networkx as nx # 3.1
from mlxtend.frequent_patterns import apriori # 0.23.1
from mlxtend.frequent_patterns import association_rules # 0.23.1

In [2]:
# remove warnings // from: https://stackoverflow.com/questions/14463277/how-to-disable-python-warnings
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
# Getting data
data = pd.read_csv("prep_data.csv")
data.head()

,Date,Client ID,ID Client Type,Cliente Type Description,ID Product,Product Description
0,2019-03-01,4442,217,Marisqueira,1,Couve Flor Mchef
1,2019-03-01,4442,217,Marisqueira,2,Espinafres Folhas Cong Masterchef
2,2019-03-01,4442,217,Marisqueira,3,Broculos Mchef Cong
3,2019-03-01,4442,217,Marisqueira,4,Grelos Amanh Cg
4,2019-03-01,4442,217,Marisqueira,5,Gel.cat.gourmes Baunilha


In [4]:
# Change types to category (THEY RESETED)
data['Date'] = pd.to_datetime(data['Date'])
data['Client ID'] = data['Client ID'].astype('category')
data['ID Client Type'] = data['ID Client Type'].astype('category')
data['Cliente Type Description'] = data['Cliente Type Description'].astype('category')
data['ID Product'] = data['ID Product'].astype('category')
data['Product Description'] = data['Product Description'].astype('category')

<hr>

<a class="anchor" id="2">

# 2. Modelling

</a>

In this section we prepared the dataset for different models that use a different version of the same dataset. This would be made in preprocessing, but it was a dinamic process and changing frequently. So it made more sense to have it in a separate section

In [5]:
# GETTING THE LIST OF PRODUCTS BOUGHT BY CLIENT PER SESSION

# Group the test dataset by Client and Date
data_transactions = data.groupby(['Client ID', 'Date','ID Client Type', 'Cliente Type Description']).apply(lambda x: x['ID Product'].tolist()).reset_index(name='ID Product')
data_transactions.dropna(inplace=True) # drop nans
data_transactions.sort_values(by=['Date', 'Client ID'], inplace=True) # sort

data_transactions.head()

,Client ID,Date,ID Client Type,Cliente Type Description,ID Product
61,448,2019-03-01,210,Hoteis 4/5 Estrelas,"[136, 607]"
126,596,2019-03-01,226,Catering Eventos,"[872, 190]"
178,648,2019-03-01,219,Rodízio Carne,"[59, 135, 137, 541, 542, 130, 263, 236, 415, 4..."
293,1013,2019-03-01,219,Rodízio Carne,"[13, 71, 49, 12, 54, 90]"
341,1275,2019-03-01,211,Cozinha Internacional,[540]


In [6]:
# SAVING DATAFRAME TO USE
data_og = data.copy()

In [7]:
# CUSTOMER-ITEM MATRIX
customer_product_matrix = data.pivot_table(
                    index='Client ID',
                    columns='ID Product',
                    values='Date',
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)

customer_product_matrix.head()

ID Product,1,2,3,4,5,6,7,8,9,10,...,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480
Client ID,,,,,,,,,,,,,,,,,,,,,
63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# DICT & FUNCTION TO GET NAMES OF IDS OF PRODUCTS
id_to_name = data.set_index('ID Product')['Product Description'].to_dict()

# Function to map IDs to product names in a list
def map_ids_to_names(id_list):
    return [id_to_name[id] for id in id_list if id in id_to_name]

<hr>

<a class="anchor" id="2_1">

## 2.1. Content-Based Filtering

</a>

Recommend products that are similar to items previously purchased by a customer. The models used in content-based recommender systems identify common features across customer preferences based on the historical purchase data of the user

In this case the similarity is based on how frequently they are bought together with other products.

In [9]:
# PRODUCT-TO-PRODUCT MATRIX
product_product_matrix = pd.DataFrame(
    pairwise_distances(customer_product_matrix.T,metric='cosine'),
    columns = customer_product_matrix.columns,
    index = customer_product_matrix.columns
    )
product_product_matrix = product_product_matrix.apply(lambda x: 1-x, axis=1) # Transform dissimilarity to similarity
product_product_matrix

ID Product,1,2,3,4,5,6,7,8,9,10,...,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480
ID Product,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.325702,0.423904,0.189934,0.043427,0.082479,0.128388,0.247436,0.116642,0.290957,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2,0.325702,1.000000,0.550885,0.259821,0.198020,0.037609,0.175629,0.188044,0.106374,0.315095,...,0.0,0.099504,0.099504,0.0,0.099504,0.0,0.000000,0.0,0.0,0.0
3,0.423904,0.550885,1.000000,0.304342,0.193293,0.036711,0.152388,0.146845,0.077876,0.453267,...,0.0,0.000000,0.000000,0.0,0.097129,0.0,0.000000,0.0,0.0,0.0
4,0.189934,0.259821,0.304342,1.000000,0.086607,0.065795,0.102418,0.131590,0.093048,0.203091,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
5,0.043427,0.198020,0.193293,0.086607,1.000000,0.263262,0.214657,0.075218,0.053187,0.066336,...,0.0,0.099504,0.099504,0.0,0.000000,0.0,0.099504,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,1.0,0.000000,0.0,0.0,0.0
2477,0.000000,0.000000,0.000000,0.000000,0.099504,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0
2478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.267261,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0


In [10]:
def get_top_similar_items(product_id, top_n=10):
    """
    Get the top N most similar items to the given product ID based on the similarity matrix.

    Parameters:
        - product_id: int or str, the ID of the product for which to find similar items.
        - top_n: int, the number of top similar items to retrieve (default is 10).

    Returns:
        - top_similar_items: list of int or str, the top N most similar product IDs.
    """
    # Retrieve the similarity scores for the given product ID
    product_similarity_scores = product_product_matrix.loc[product_id]

    # Sort the similarity scores in descending order and exclude the product itself
    top_similar_items = product_similarity_scores.sort_values(ascending=False).iloc[1:top_n+1].index.tolist()

    return top_similar_items

# Example usage
product_id = 2000
top_10_similar_items = get_top_similar_items(product_id)
print("Top 10 similar items for product", product_id, ":", top_10_similar_items)


Top 10 similar items for product 2000 : [1938, 2181, 2423, 1396, 1774, 1638, 2459, 2458, 1566, 2384]


In [11]:
# Save list of products
product_list_cbr = pd.DataFrame(data_og[["ID Product","Product Description"]].drop_duplicates()).reset_index(drop=True)
product_list_cbr['ID Product'] = product_list_cbr['ID Product'].to_list()

# Apply function
product_list_cbr["Similar Recommendations"] = product_list_cbr['ID Product'].apply(get_top_similar_items)

# Apply the mapping function to the recommendation lists column
product_list_cbr['Recommendation Names'] = product_list_cbr['Similar Recommendations'].apply(map_ids_to_names)

product_list_cbr


,ID Product,Product Description,Similar Recommendations,Recommendation Names
0,1,Couve Flor Mchef,"[3, 67, 1116, 1576, 55, 1385, 63, 2, 983, 45]","[Broculos Mchef Cong, Couve Bruxelas Cong Mast..."
1,2,Espinafres Folhas Cong Masterchef,"[3, 63, 204, 577, 334, 276, 12, 273, 331, 139]","[Broculos Mchef Cong, Ervilha Mchef Cong, Açúc..."
2,3,Broculos Mchef Cong,"[2, 63, 10, 1, 204, 99, 45, 12, 199, 334]","[Espinafres Folhas Cong Masterchef, Ervilha Mc..."
3,4,Grelos Amanh Cg,"[191, 190, 3, 167, 63, 800, 80, 303, 2276, 1061]","[Azeitonas Pretas S/caroço Coopoliva, Vinho Fr..."
4,5,Gel.cat.gourmes Baunilha,"[15, 16, 14, 488, 21, 560, 507, 173, 204, 175]","[Gel.cat.gourmes Morango, Gel.cat.gourmes, Gel..."
...,...,...,...,...
2475,2476,Creme Chocolate Leite Monin,"[1192, 566, 1287, 476, 883, 695, 831, 1106, 31...","[Cerv C/alc Cergal, Bolacha Rech Oreo Tubo Ori..."
2476,2477,Granola Serramel,"[2161, 2244, 1750, 2268, 1435, 2333, 1511, 228...","[Besugo Grs.(cx, Pampo, Maçã Reineta Rch, Agua..."
2477,2478,Linguado Medio Rch,"[2478, 2259, 2303, 2154, 1073, 2233, 1966, 132...","[Linguado Medio Rch, Cantaril Kg.(cx, Faneca, ..."
2478,2479,Pregado Aquicultura Plt,"[1983, 199, 1, 1658, 1652, 1653, 1654, 1655, 1...","[Pregado Rch, Pickles Guloso, Couve Flor Mchef..."


* For this model we took the approach of offering a unique reccomendation based on the product in question (not the client's history). The ideia is to offer this reccomendation for each product when selecting or choosing it online.

<hr>

<a class="anchor" id="2_2">

## 2.2. Collaborative Filtering

</a>

"Collaborative filtering is a technique that can filter out items that a user might like on the basis of reactions by similar users. It works by searching a large group of people and finding a smaller set of users with tastes similar to a particular user. It looks at the items they like and combines them to create a ranked list of suggestions."

In [12]:
def get_top_recommendations_coll(measure, client, top_n=10):
    # Find top similar clients
    similar_clients = measure[customer_product_matrix.index.get_loc(client)].argsort()[::-1][1:]

    # Get products purchased by similar clients but not by the target client
    similar_products_mask = customer_product_matrix.iloc[similar_clients].values == 1
    target_products_mask = customer_product_matrix.loc[client].values == 0
    new_products_mask = similar_products_mask & target_products_mask

    # Count the frequency of new products
    new_products_count = new_products_mask.sum(axis=0)

    # Sort the new products by frequency and return top N
    top_indices = new_products_count.argsort()[::-1][:top_n]
    top_recommendations = customer_product_matrix.columns[top_indices].tolist()

    return top_recommendations


def use_recommendation_coll(row):
    """
    Get Recommendations for each client
    """
    client_id = row["Client ID"]
    recommendations = get_top_recommendations_coll(client_similarity_jaccard, client_id) # Change with best measure
    return recommendations

* As for similarity measures, we only took into account jaccard and cosine, since usually they  are the best for binary matrices

* To choose the best similarity measure we took into account the running time of each 

In [13]:
# Calculate client similarity
client_similarity_cosine = cosine_similarity(customer_product_matrix)

cosine_time = []
# Example usage COSINE
for i in range(1000):
    start_time = time.time()
    client = 1275
    top_recommendations = get_top_recommendations_coll(client_similarity_cosine,client)
    end_time = time.time()
    cosine_time.append((end_time-start_time))
    
print("Top 10 products based on similar clients for client ", client, ":", top_recommendations, "\nAverage Time:", np.mean(cosine_time), "seconds")

Top 10 products based on similar clients for client  1275 : [136, 210, 173, 299, 12, 203, 168, 160, 189, 147] 
Average Time: 0.013975059986114502 seconds


In [14]:
# Calculate client similarity
client_similarity_jaccard = 1 - pairwise_distances(customer_product_matrix.values, metric='jaccard')

jaccard_time = []
# Example usage
for i in range(1000):
    start_time = time.time()
    client = 1275
    top_recommendations = get_top_recommendations_coll(client_similarity_jaccard,client)
    end_time = time.time()
    jaccard_time.append((end_time-start_time))
    
print("Top 10 products based on similar clients for client ", client, ":", top_recommendations, "\nAverage Time:", np.mean(jaccard_time), "seconds")

Top 10 products based on similar clients for client  1275 : [136, 210, 173, 299, 12, 203, 168, 160, 189, 147] 
Average Time: 0.01641117525100708 seconds


* Even choosing 1000 iterations to test the running time, the Average times oscilates a lot still. And the best measure keeps changing between the two. For this reason we kept using cosine, for this and future algorithms

In [15]:
# Save list of products
client_list_coll = data_transactions[["Client ID", "Cliente Type Description" , "ID Product"]].reset_index(drop=True)
client_list_coll["Client ID"].drop_duplicates(inplace=True)

# Save list of products
client_list_coll["Recommendations IDs"] = client_list_coll.apply(use_recommendation_coll, axis=1)

client_list_coll["Recommendations Name"] = client_list_coll["Recommendations IDs"].apply(map_ids_to_names)

In [16]:
client_list_coll.head()

,Client ID,Cliente Type Description,ID Product,Recommendations IDs,Recommendations Name
0,448,Hoteis 4/5 Estrelas,"[136, 607]","[276, 152, 139, 173, 299, 206, 12, 203, 168, 160]","[Leite Mchef Uht M/gordo, Farinha Mchef S/ferm..."
1,596,Catering Eventos,"[872, 190]","[276, 152, 210, 139, 173, 273, 299, 206, 203, ...","[Leite Mchef Uht M/gordo, Farinha Mchef S/ferm..."
2,648,Rodízio Carne,"[59, 135, 137, 541, 542, 130, 263, 236, 415, 4...","[136, 152, 210, 139, 173, 273, 299, 206, 12, 168]","[Vinagre Br Chef, Farinha Mchef S/fermento, Po..."
3,1013,Rodízio Carne,"[13, 71, 49, 12, 54, 90]","[152, 139, 273, 206, 203, 147, 296, 199, 443, ...","[Farinha Mchef S/fermento, Óleo Alimentar Mche..."
4,1275,Cozinha Internacional,[540],"[136, 210, 173, 299, 12, 203, 168, 160, 189, 147]","[Vinagre Br Chef, Polpa Tomate Mastercheff, De..."


* As one of models with the highest time running, it was a bit limited in what we could do in order to not go for hours running.

<hr>

<a class="anchor" id="2_3">

## 2.3. Popularity-Based Recommendations

</a>

Popularity based recommendation system works with the trend. It basically uses the items which are in trend right now. Here is an example for Popularity based recommendation system with Python code.

* Since the average products bought per transiction is 13.302891 we will recommend 15 products
* Popularity based on the last month available of sales

SOURCES:

https://github.com/LaxmiChaudhary/Amzon-Product-Recommendation/tree/master

https://towardsdatascience.com/evaluation-metrics-for-recommendation-systems-an-overview-71290690ecba

CHATGPT 3.5

In [17]:
# SPLIT TRAIN & TEST

#Split the data into train and test datasets into 80:20 ratio
train_data, test_data = train_test_split(data_transactions, test_size = 0.2, random_state=0, shuffle=False)

In [18]:
# Flatten the lists of products
train_data_exp = train_data.explode('ID Product')
test_data_exp = test_data.explode('ID Product')

* Since these models were based on previous purchases, with goal of trying to predict based on popularity, we were able to evaluate the results
* The construction made for the popularity based recommendation made it difficult to use K-Fold techniques
* Since we wanted to mantain the transaction date order, in order to simulate new customers transaction it ruled out anything with random shuffle like basic K-Fold or Monte Carlos Cross Validation
* Additionaly, we tested timeseries split  methods, but the running times grew exponential (so we ruled out as an option)

<hr>

<a class="anchor" id="2_3_1">

### 2.3.1. General Popularity

</a>

In [19]:
def evaluate_model_popularity_general(data, purchase, top):
    """
    Evaluate the model performance based on precision, F1 score, and hit rate.
    
    """
    # Initialize lists to store individual metric scores
    precision_scores = []
    f1_scores = []
    hit_rates = []

    # Iterate over each row of the DataFrame
    for index, row in data.iterrows():
        purchase_list = set(row[purchase])
        top_items_set = set(top)
        
        # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN)
        TP = len(purchase_list & top_items_set)  # Both purchased and in top items
        FP = len(top_items_set - purchase_list)  # In top items but not purchased
        FN = len(purchase_list - top_items_set)  # Purchased items but not in top items
        
        # Calculate Precision
        if TP + FP == 0:
            precision = 0  # Handle division by zero
        else:
            precision = TP / (TP + FP)
        precision_scores.append(precision)
        
        # Calculate Recall
        if TP + FN == 0:
            recall = 0  # Handle division by zero
        else:
            recall = TP / (TP + FN)
        
        # Calculate F1 Score
        if precision + recall == 0:
            f1 = 0  # Handle division by zero
        else:
            f1 = 2 * (precision * recall) / (precision + recall)
        f1_scores.append(f1)
        
        # Calculate hit rate
        hit_rate = 1 if TP > 0 else 0
        hit_rates.append(hit_rate)

    # Calculate average metrics
    avg_precision = sum(precision_scores) / len(precision_scores)
    avg_f1 = sum(f1_scores) / len(f1_scores)
    avg_hit_rate = sum(hit_rates) / len(hit_rates)
    
    return avg_precision, avg_f1, avg_hit_rate

In [20]:
# GETTING TOP ITEMS

# Compute the start date and end date for the last month
end_date = train_data_exp['Date'].max()  # Since the last day of may is 31
start_date = end_date - timedelta(days=30)  

# Filter the dataset to include only transactions from the last month
last_month_data = train_data_exp[(train_data_exp['Date'] >= start_date) & (train_data_exp['Date'] <= end_date)]

##########################################################################
# The number of days can be adjusted based on the goal of how recent the data should be
# For example, if a different timeframe is desired, modify the timedelta value accordingly
# The timedelta(days=30) represents a month, but this can be changed based on requirements
##########################################################################

# Compute item popularity based on the number of times is bought
item_popularity = last_month_data['ID Product'].value_counts().reset_index()
item_popularity.columns = ['ID Product', 'Popularity']

# Sort items by popularity
item_popularity = item_popularity.sort_values(by='Popularity', ascending=False)

# Defining the number of recommended items
top_n = 10

# Get the top N recommended items
top_items = item_popularity.head(top_n)['ID Product'].tolist()

In [21]:
train_precision, train_f1, train_hit_rate = evaluate_model_popularity_general(train_data, 'ID Product', top_items)
test_precision, test_f1, test_hit_rate = evaluate_model_popularity_general(test_data, 'ID Product', top_items)

print("========TRAIN========")
print("Precision: ", round(train_precision, 4), "\nF1: ", round(train_f1, 4), "\nHit Rate: ", round(train_hit_rate, 4))

print("\n========TEST========")
print("Precision: ", round(test_precision, 4), "\nF1: ", round(test_f1, 4), "\nHit Rate: ", round(test_hit_rate, 4))

========TRAIN========
Precision:  0.1104 
F1:  0.0784 
Hit Rate:  0.5412

========TEST========
Precision:  0.1088 
F1:  0.078 
Hit Rate:  0.5335


In [22]:
# RECOMMENDATIONS NAMES
map_ids_to_names(top_items)

['Açúcar Mchef Branco Papel',
 'Leite Mchef Uht M/gordo',
 'Óleo Alimentar Mchef',
 'Alface Frisada Plt',
 'Det Loica Mchef',
 'Vinagre Br Chef',
 'Farinha Mchef S/fermento',
 'Ovo Agro Ovo Ind',
 'Ovo Agro Ovo Ind',
 'Coentros Plt']

*"Since, it is a Popularity recommender model, so, all the three users are given the same recommendations. Here, we predict the products based on the popularity. It is not personalized to particular user. It is a non-personalized recommender system."*

<hr>

<a class="anchor" id="2_3_2">

### 2.3.2. Popularity per Type of Client

</a>

In [23]:
def recommend_popular_items_type_client(client_type, top_n=10):
    """
    Get the list of recommendations for one client
    
    We added the option to filter the last 30 days of data, this way recency can be accounted
    Ultimately it works better with more data, 1 year or 6 months, rather than just 1 month
    for this reason we will leave as commentary
    
    """
    # Compute the start date and end date for the last month
    # end_date = train_data_exp['Date'].max()  # Determine the latest date in the dataset
    # start_date = end_date - timedelta(days=30)  # Assuming 30 days in a month 

    # Filter client's purchase history for the last month
    user_history = train_data_exp[(train_data_exp['ID Client Type'] == client_type) 
                                  # & (train_data_exp['Date'] >= start_date) & 
                                  # (train_data_exp['Date'] <= end_date)
                                 ]
    # Group by product ID and count occurrences
    popular_items = user_history.groupby('ID Product').size().reset_index(name='purchase_count')

    # Sort by purchase count in descending order
    popular_items = popular_items.sort_values(by='purchase_count', ascending=False)

    # Get top N recommended items
    top_recommendations = popular_items.head(top_n)['ID Product'].tolist()

    return top_recommendations

def get_recommendations_type_client(row):
    """
    Get Recommendations for each client

    """
    client_id = row['ID Client Type']
    
    # If client ID is in the test set
    if client_id in train_data_exp['ID Client Type'].unique():
        recommendations = recommend_popular_items_type_client(client_id)
    else:
        # Use top list
        recommendations = top_items # from previous algorithm
        
    return recommendations

def evaluate_model_popularity_client(data, purchase, recommendation):
    """
    Evaluate the model performance based on precision, F1 score, and hit rate.

    """
    # Initialize lists to store individual metric scores
    precision_scores = []
    f1_scores = []
    hit_rates = []

    # Iterate over each row of the DataFrame
    for index, row in data.iterrows():
        purchase_list = set(row[purchase])
        top_items_set = set(row[recommendation])
        
        # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN)
        TP = len(purchase_list & top_items_set)  # Both purchased and in top items
        FP = len(top_items_set - purchase_list)  # In top items but not purchased
        FN = len(purchase_list - top_items_set)  # Purchased items but not in top items
        
        # Calculate Precision
        if TP + FP == 0:
            precision = 0  # Handle division by zero
        else:
            precision = TP / (TP + FP)
        precision_scores.append(precision)
        
        # Calculate Recall
        if TP + FN == 0:
            recall = 0  # Handle division by zero
        else:
            recall = TP / (TP + FN)
        
        # Calculate F1 Score
        if precision + recall == 0:
            f1 = 0  # Handle division by zero
        else:
            f1 = 2 * (precision * recall) / (precision + recall)
        f1_scores.append(f1)
        
        # Calculate hit rate
        hit_rate = 1 if TP > 0 else 0
        hit_rates.append(hit_rate)

    # Calculate average metrics
    avg_precision = sum(precision_scores) / len(precision_scores)
    avg_f1 = sum(f1_scores) / len(f1_scores)
    avg_hit_rate = sum(hit_rates) / len(hit_rates)
    
    return avg_precision, avg_f1, avg_hit_rate

In [24]:
# DATASET FOR EVALUATION

# for train
train_type_client = train_data_exp.groupby(['Date','Client ID','ID Client Type','Cliente Type Description'])['ID Product'].apply(list).reset_index()
train_type_client.dropna(inplace=True) # drop nans
train_type_client.sort_values(by=['Date', 'Client ID','ID Client Type'], inplace=True) # sort
train_type_client['Recommendations'] = train_type_client.apply(get_recommendations_type_client, axis=1)

# for test
test_type_client = test_data_exp.groupby(['Date','Client ID','ID Client Type', 'Cliente Type Description'])['ID Product'].apply(list).reset_index()
test_type_client.dropna(inplace=True) # drop nans
test_type_client.sort_values(by=['Date', 'Client ID'], inplace=True) # sort
test_type_client['Recommendations'] = test_type_client.apply(get_recommendations_type_client, axis=1)

In [25]:
# EVALUATION

train_precision, train_f1, train_hit_rate = evaluate_model_popularity_client(train_type_client, 'ID Product', 'Recommendations')
test_precision, test_f1, test_hit_rate = evaluate_model_popularity_client(test_type_client, 'ID Product', 'Recommendations')

print("========TRAIN========")
print("Precision: ", round(train_precision, 4), "\nF1: ", round(train_f1, 4), "\nHit Rate: ", round(train_hit_rate, 4))

print("\n========TEST========")
print("Precision: ", round(test_precision, 4), "\nF1: ", round(test_f1, 4), "\nHit Rate: ", round(test_hit_rate, 4))

========TRAIN========
Precision:  0.1802 
F1:  0.1427 
Hit Rate:  0.6753

========TEST========
Precision:  0.1667 
F1:  0.1326 
Hit Rate:  0.6519


In [26]:
# Apply the mapping function to the recommendation lists column
train_type_client['Recommendation Names'] = train_type_client['Recommendations'].apply(map_ids_to_names)
test_type_client['Recommendation Names'] = test_type_client['Recommendations'].apply(map_ids_to_names)

test_type_client[['Cliente Type Description','Recommendation Names']].drop_duplicates("Cliente Type Description").reset_index(drop=True)


,Cliente Type Description,Recommendation Names
0,Cozinha Portuguesa,"[Açúcar Mchef Branco Papel, Leite Mchef Uht M/..."
1,Cozinha Japonesa,"[Açúcar Mchef Branco Papel, Pepino Rch, Manga ..."
2,Cozinha Chinesa,"[Açúcar Mchef Branco Papel, Couve Coracao De B..."
3,Cozinha Italiana,"[Atum Maresia Óleo Bolsa, Ananas Guloso Pedaco..."
4,Cozinha Internacional,"[Açúcar Mchef Branco Papel, Óleo Alimentar Mch..."
5,Churrasqueira,"[Óleo Alimentar Mchef, Cerv C/alc Sagres, Alfa..."
6,Especialidade Peixe,"[Óleo Alto Rendimento Cintra, Grelos Amanh Cg,..."
7,Tapas & Petiscos,"[Ovo Agro Ovo Ind, Presunto Fatias Masterchef,..."
8,Hoteis 4/5 Estrelas,"[Açúcar Mchef Branco Papel, Queijo Flamengo Vi..."
9,Catering Eventos,"[Queijo Flamengo Villas Fat, Fiambre Pa Mchef ..."


<hr>

<a class="anchor" id="2_3_3">

### 2.3.3. Popularity per Client

</a>

In [27]:
def recommend_popular_items(client, top_n=10):
    """
    Get the list of recommendations for one client
    
    We added the option to filter the last 30 days of data, this way recency can be accounted
    Ultimately it works better with more data, 1 year or 6 months, rather than just 1 month
    for this reason we will leave as commentary
    
    """
    # Compute the start date and end date for the last month
    # end_date = train_data_exp['Date'].max()  # Determine the latest date in the dataset
    # start_date = end_date - timedelta(days=30)  # Assuming 30 days in a month 

    # Filter client's purchase history for the last month
    user_history = train_data_exp[(train_data_exp['Client ID'] == client) 
                                  # & (train_data_exp['Date'] >= start_date) & 
                                  # (train_data_exp['Date'] <= end_date)
                                 ]

    # Group by product ID and count occurrences
    popular_items = user_history.groupby('ID Product').size().reset_index(name='purchase_count')

    # Sort by purchase count in descending order
    popular_items = popular_items.sort_values(by='purchase_count', ascending=False)

    # Get top N recommended items
    top_recommendations = popular_items.head(top_n)['ID Product'].tolist()

    return top_recommendations

def get_recommendations(row):
    """
    Get Recommendations for each client

    """
    client_id = row['Client ID']
    
    # If client ID is in the test set
    if client_id in train_data_exp['Client ID'].unique():
        recommendations = recommend_popular_items(client_id)
    else:
        # Use top list
        recommendations = top_items # from previous algorithm
        
    return recommendations

In [28]:
# DATASET FOR EVALUATION

# for train
train_data_client = train_data_exp.groupby(['Date','Client ID'])['ID Product'].apply(list).reset_index()
train_data_client.dropna(inplace=True) # drop nans
train_data_client.sort_values(by=['Date', 'Client ID'], inplace=True) # sort
train_data_client['Recommendations'] = train_data_client.apply(get_recommendations, axis=1)

# for test
test_data_client = test_data_exp.groupby(['Date','Client ID'])['ID Product'].apply(list).reset_index()
test_data_client.dropna(inplace=True) # drop nans
test_data_client.sort_values(by=['Date', 'Client ID'], inplace=True) # sort
test_data_client['Recommendations'] = test_data_client.apply(get_recommendations, axis=1)

In [29]:
# EVALUATION

train_precision, train_f1, train_hit_rate = evaluate_model_popularity_client(train_data_client, 'ID Product', 'Recommendations')
test_precision, test_f1, test_hit_rate = evaluate_model_popularity_client(test_data_client, 'ID Product', 'Recommendations')

print("========TRAIN========")
print("Precision: ", round(train_precision, 4), "\nF1: ", round(train_f1, 4), "\nHit Rate: ", round(train_hit_rate, 4))

print("\n========TEST========")
print("Precision: ", round(test_precision, 4), "\nF1: ", round(test_f1, 4), "\nHit Rate: ", round(test_hit_rate, 4))

========TRAIN========
Precision:  0.5479 
F1:  0.4572 
Hit Rate:  0.9366

========TEST========
Precision:  0.4701 
F1:  0.3925 
Hit Rate:  0.9001


In [30]:
train_data_client['Recommendation Names'] = train_data_client['Recommendations'].apply(map_ids_to_names)
test_data_client['Recommendation Names'] = test_data_client['Recommendations'].apply(map_ids_to_names)

# Final dataset
test_data_client[['Client ID','Recommendation Names']].drop_duplicates("Client ID").reset_index(drop=True)

,Client ID,Recommendation Names
0,7971,"[Azeite Gallo Subtil Poupança, Det Loica Mchef..."
1,8021,"[Ref Coca Cola Zero, Ref Coca Cola Original, R..."
2,8025,"[Frango Peito Cong Saco, Batata Conservação As..."
3,8041,"[Broculos Mchef Cong, Polpa Tomate Mastercheff..."
4,8049,"[Cogumelo Branco Míudo Rch, Óleo Alimentar Mch..."
...,...,...
779,7720,"[Leite Condensado Masterchef, Leite Mchef Uht ..."
780,7775,"[Paloco Pacifico Desf Cong Gran.cx, Filetes De..."
781,8348,"[Óleo Fula Profissional, Farinha Mchef S/ferme..."
782,8427,"[Doce Bonne Maman, Doce Bonne Maman, Doce Bonn..."


<hr>

<a class="anchor" id="2_3_4">

### 2.3.4. "Did you forget?" approach

</a>

* Our idea for "Did you Forget?" approach at the check-out moment was to use the popularity based reccommendation for each client, and remove any product that is already on the basket, filling it with the next one in line

In [31]:
def suggest_forgotten_items(train_data, transaction, top_n=10):
    """
    Suggest products based on popularity that are not present in the client's basket for a single transaction.
    """
    # Get items in the client's basket from this transaction
    basket_items = transaction['ID Product']

    # Filter train data to include only the purchases of the specific customer
    customer_train_data = train_data[train_data['Client ID'] == transaction['Client ID']]

    # Count occurrences of each product in the train data of the specific customer
    product_counts = customer_train_data['ID Product'].explode().value_counts()

    # Get top N recommended items not in client's basket for this transaction
    suggested_items = []
    for product in product_counts.index:
        if len(suggested_items) >= top_n:
            break
        if product not in basket_items:
            suggested_items.append(product)

    return suggested_items

# Recommendation Results
forget_rec_data = test_data.copy()

# Apply the function to each row of test_data and add the recommendations as a new column
forget_rec_data['Recommendations'] = forget_rec_data.apply(lambda x: suggest_forgotten_items(train_data, x), axis=1)

# Display the updated test_data DataFrame
forget_rec_data



,Client ID,Date,ID Client Type,Cliente Type Description,ID Product,Recommendations
10777,7971,2019-05-15,201,Cozinha Portuguesa,"[34, 76, 1501, 40, 95, 204, 206, 136, 273, 373...","[157, 278, 1395, 812, 1483, 324, 1468, 1089, 4..."
10961,8021,2019-05-15,214,Cozinha Japonesa,"[1310, 1590, 805, 299, 203]","[296, 454]"
11011,8025,2019-05-15,201,Cozinha Portuguesa,"[115, 58, 807, 376, 379, 664, 240, 540, 168, 5...","[663, 38, 377, 1346, 887, 2293, 345, 1645, 737..."
11084,8041,2019-05-15,203,Cozinha Chinesa,"[3, 75, 204, 175, 177, 136, 328, 210, 814, 879...","[878, 282, 320, 353, 427, 2047, 206, 330, 2290..."
11166,8049,2019-05-15,214,Cozinha Japonesa,"[2388, 87, 63, 11, 5, 14, 16, 552, 130, 231, 2...","[270, 345, 15, 232, 233, 351, 331, 173, 27, 350]"
...,...,...,...,...,...,...
17309,8872,2019-05-31,201,Cozinha Portuguesa,"[2431, 1103, 2383, 434, 389]","[1798, 1861, 1531]"
17320,8876,2019-05-31,228,Hoteis 3 Estrelas,"[2416, 2405]",[]
17333,8880,2019-05-31,227,Bares/snacks/cafes/gelatarias/brunch,"[71, 50, 51, 12, 505, 238, 1111, 143, 299]",[]
17353,8892,2019-05-31,214,Cozinha Japonesa,"[423, 250, 251, 698, 254, 377, 808, 1196, 760,...",[]


* For the construction of the algorithm we used train data as historical data, and the test section as a simulation of new clients and what they would view as reccommendations

In [32]:
# Apply the mapping function to the recommendation lists column
forget_rec_data['Recommendation Names'] = forget_rec_data['Recommendations'].apply(map_ids_to_names)
forget_rec_data['Basket Names'] = forget_rec_data['ID Product'].apply(map_ids_to_names)

# Final dataset
forget_rec_data[['Client ID','Date','Basket Names','Recommendation Names']].reset_index(drop=True)

,Client ID,Date,Basket Names,Recommendation Names
0,7971,2019-05-15,"[Bacalhau Lombo Ps Iqf Cg, Miolo Camarao Cg, C...","[Leite Est.atlantico Uht M/g, Manteiga Mimosa ..."
1,8021,2019-05-15,"[Camarão Bt Cg, Arroz Carolino Masterchef, Ref...","[Água C/gas Pedras Salgadas, Massa Mchef Fusil..."
2,8025,2019-05-15,"[Pastel De Nata Mchef, Frango Peito Cong Saco,...","[Batata Conservação Assar Verm Rch, Camarao S/..."
3,8041,2019-05-15,"[Broculos Mchef Cong, Porco Lombo S/o Cong C/ ...","[Bovino Maminha, Bolacha Amanhecer Maria, Vaqu..."
4,8049,2019-05-15,"[Bat. Pal. Ecofrost Cong, Sob. Tranche Bolacha...","[Bacon Fatiado Masterchef, Cebola Rch, Gel.cat..."
...,...,...,...,...
3468,8872,2019-05-31,"[Bov.picanha Europa Aa Cong.kg, Bovino Maminha...","[Carapau Medio Rch, Dourada Amanhada, Carvao V..."
3469,8876,2019-05-31,"[Mamão Descascado Cubos Rch, Salada Fruta Em C...",[]
3470,8880,2019-05-31,"[Polpa Fruta Cong Morango Brasfrut, Polpa Frut...",[]
3471,8892,2019-05-31,"[Gelatina Folhas Jeronimos, Manga Barco Cal Rc...",[]


* If it is a first time customer it will not receive any recommendation!
* The list of recommendations can be less than 10 in cases where the customer already has most of his popular products

<hr>

<a class="anchor" id="2_5">

## 2.4. Smart Basket Recommendation

</a>

The approach is based on the theory that customers who buy a certain item (or group of items) are more likely to buy another specific item (or group of items). 

SOURCES:
    
https://365datascience.com/tutorials/python-tutorials/market-basket-analysis/

In [33]:
# Function to replace every value above 1 as 1
def above1_to1(item_freq):
    res = 0
    if item_freq > 0:
        res = 1
    return res

In [34]:
# PREPARATION OF DATASET

# Copy section of data
data_sbs = data[['Date', 'Client ID', 'ID Product']]

# Create a column to identify unique transactions (per date and client)
data_sbs['Single Transaction'] = data_sbs['Client ID'].astype(str)+' - '+data_sbs['Date'].astype(str)

# Pivot table to have count of products per transaction
sbs_pivot = pd.crosstab(data_sbs['Single Transaction'], data_sbs['ID Product'])

# Use function, since Smart Basket does not identify frequency
smartbasket_input = sbs_pivot.applymap(above1_to1)

smartbasket_input.head()

ID Product,1,2,3,4,5,6,7,8,9,10,...,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480
Single Transaction,,,,,,,,,,,,,,,,,,,,,
1013 - 2019-03-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1013 - 2019-03-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1013 - 2019-03-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1013 - 2019-03-12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1013 - 2019-03-15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# DEFINING MIN SUPPORT

# Count the number of times each product appears
product_counts = data_sbs['ID Product'].value_counts()

# Calculate the percentage of each product
product_percentages = (product_counts / data_sbs["Single Transaction"].nunique()) * 100
# 
# Create a DataFrame to store the counts and percentages
product_summary = pd.DataFrame({'Product': product_counts.index, 
                                'Count': product_counts.values,
                                'Percentage': product_percentages.values})

display(product_summary)

display(product_summary.describe(percentiles=[0.86]).T)

,Product,Count,Percentage
0,204,3354,19.319164
1,139,2781,16.018663
2,276,2413,13.898969
3,379,1835,10.569668
4,173,1686,9.711422
...,...,...,...
2475,2336,1,0.005760
2476,1554,1,0.005760
2477,2341,1,0.005760
2478,1558,1,0.005760


,count,mean,std,min,50%,86%,max
Count,2480.0,93.325403,197.829985,1.00000,26.000000,180.000000,3354.000000
Percentage,2480.0,0.537558,1.139508,0.00576,0.149761,1.036807,19.319164


* We tried to define a minimum support that would make sense for the context of the data, and in a way that code could run without any problem
* With this in mind we chose 1.1%, where only around 14% of products will be appearing on the algorithm

In [36]:
# Apply the Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(smartbasket_input, min_support=0.011, use_colnames=True)
# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(2),(3),0.030701,0.056506,0.011750,0.382739,6.773431,0.010016,1.528518,0.879362
1,(3),(2),0.056506,0.030701,0.011750,0.207951,6.773431,0.010016,1.223787,0.903413
2,(3),(12),0.056506,0.072346,0.014458,0.255861,3.536632,0.010370,1.246614,0.760201
3,(12),(3),0.072346,0.056506,0.014458,0.199841,3.536632,0.010370,1.179133,0.773182
4,(3),(139),0.056506,0.160187,0.019642,0.347604,2.169997,0.010590,1.287276,0.571461


In [37]:
def suggest_consequent_items(rules_df, basket_items):
    """
    Suggest consequent items based on the association rules if their antecedents are present in the basket.
    """
    # Convert basket_items to a set for faster membership checking
    basket_items_set = set(basket_items)
    
    # Check if any of the antecedent items are present in the basket
    antecedents_present = rules_df['antecedents'].apply(lambda x: bool(set(x) & basket_items_set))
    
    # Get the consequent items corresponding to the rules where antecedents are present in the basket
    consequents_suggested = rules_df.loc[antecedents_present, 'consequents'].explode().unique()
    
    return consequents_suggested.tolist()

# Example usage with apply
def suggest_consequent_items_for_row(row):
    basket_items = row['ID Product']  # Assuming 'ID Product' is the column containing the basket items
    suggestions = suggest_consequent_items(rules, basket_items)
    return suggestions


In [38]:
recom_smart_basket = data_transactions.copy()

# Apply the function to each row of your DataFrame
recom_smart_basket['Recommended_Items'] = recom_smart_basket.apply(suggest_consequent_items_for_row, axis=1)

# Display the updated DataFrame with recommended items
recom_smart_basket

,Client ID,Date,ID Client Type,Cliente Type Description,ID Product,Recommended_Items
61,448,2019-03-01,210,Hoteis 4/5 Estrelas,"[136, 607]","[139, 152, 168, 173, 175, 177, 204, 206, 210, ..."
126,596,2019-03-01,226,Catering Eventos,"[872, 190]",[]
178,648,2019-03-01,219,Rodízio Carne,"[59, 135, 137, 541, 542, 130, 263, 236, 415, 4...","[3, 12, 139, 173, 177, 204, 210, 237, 276, 130..."
293,1013,2019-03-01,219,Rodízio Carne,"[13, 71, 49, 12, 54, 90]","[3, 139, 161, 173, 204, 276, 379]"
341,1275,2019-03-01,211,Cozinha Internacional,[540],[]
...,...,...,...,...,...,...
17309,8872,2019-05-31,201,Cozinha Portuguesa,"[2431, 1103, 2383, 434, 389]",[]
17320,8876,2019-05-31,228,Hoteis 3 Estrelas,"[2416, 2405]",[]
17333,8880,2019-05-31,227,Bares/snacks/cafes/gelatarias/brunch,"[71, 50, 51, 12, 505, 238, 1111, 143, 299]","[3, 139, 161, 173, 204, 276, 130, 203, 237, 29..."
17353,8892,2019-05-31,214,Cozinha Japonesa,"[423, 250, 251, 698, 254, 377, 808, 1196, 760,...","[204, 232, 233, 237, 252, 699, 255, 379, 378]"


In [39]:
# Apply the mapping function to the recommendation lists column
recom_smart_basket['Recommendation Names'] = recom_smart_basket['Recommended_Items'].apply(map_ids_to_names)
recom_smart_basket['Basket Names'] = recom_smart_basket['ID Product'].apply(map_ids_to_names)

# Final dataset
recom_smart_basket[['Client ID','Date','Basket Names','Recommendation Names']].reset_index(drop=True)

,Client ID,Date,Basket Names,Recommendation Names
0,448,2019-03-01,"[Vinagre Br Chef, Bebida Soja Vive Soy Natural]","[Óleo Alimentar Mchef, Farinha Mchef S/ferment..."
1,596,2019-03-01,"[Cogumelos Inteiros Mast.chef, Vinho Frades Bco]",[]
2,648,2019-03-01,"[Salada Russa Cong Mchef, Arroz Cozinha Velha ...","[Broculos Mchef Cong, Frutos Vermelhos Mchef, ..."
3,1013,2019-03-01,"[Bat. Pal. Mchef Cong, Polpa Fruta Cong Morang...","[Broculos Mchef Cong, Óleo Alimentar Mchef, At..."
4,1275,2019-03-01,[Dourada Rch],[]
...,...,...,...,...
17356,8872,2019-05-31,"[Bov.picanha Europa Aa Cong.kg, Bovino Maminha...",[]
17357,8876,2019-05-31,"[Mamão Descascado Cubos Rch, Salada Fruta Em C...",[]
17358,8880,2019-05-31,"[Polpa Fruta Cong Morango Brasfrut, Polpa Frut...","[Broculos Mchef Cong, Óleo Alimentar Mchef, At..."
17359,8892,2019-05-31,"[Gelatina Folhas Jeronimos, Manga Barco Cal Rc...","[Açúcar Mchef Branco Papel, Pepino Rch, Morang..."


* Unfortenely, as mention, only 14% of the products received an association, so customers that did not purchase any of these product will not receive any reccomendation

<hr>

<a class="anchor" id="2_5">

## 2.5. Page Rank Recommendation
    
</a>


PageRank is an algorithm used to measure the importance of nodes in a network by considering the quantity and quality of links to them.

In [40]:
data_pagerank_train = train_data[["Client ID","Date","ID Product"]]

# Create a directed graph
G = nx.DiGraph()

# Add edges representing product co-occurrences in transactions
edges = train_data['ID Product'].explode().groupby(level=0).apply(lambda x: list(zip(x[:-1], x[1:])))
G.add_edges_from([edge for sublist in edges for edge in sublist])

# Calculate PageRank scores
pagerank_scores = nx.pagerank(G)

# Recommend products based on PageRank scores
def recommend_products(client, num_recommendations=10):
    # Check if the client has a transaction history
    if client not in train_data["Client ID"].unique():
        # If the client has no transaction history, provide generalized recommendations
        generalized_recommendations = sorted(pagerank_scores.keys(), key=pagerank_scores.get, reverse=True)
        return generalized_recommendations[:num_recommendations]
    
    # Get the client's transaction history
    client_products_train = train_data[train_data["Client ID"] == client]["ID Product"].explode().unique()
    
    # Calculate personalized recommendations based on PageRank scores
    personalized_recommendations = sorted(client_products_train, key=lambda x: pagerank_scores.get(x, 0), reverse=True)
    
    # Check if the number of personalized recommendations is less than 10
    if len(personalized_recommendations) < num_recommendations:
        # If the number of personalized recommendations is less than 10, fill the rest with generalized recommendations
        remaining_recommendations = num_recommendations - len(personalized_recommendations)
        generalized_recommendations = [product for product in pagerank_scores.keys() if product not in personalized_recommendations]
        personalized_recommendations.extend(generalized_recommendations[:remaining_recommendations])
    
    return personalized_recommendations[:num_recommendations]


* We decided to opt for an approach that leverages the co-occurrence of products in transactions and the PageRank algorithm to provide personalized recommendations to clients. 

In [41]:
# TRAIN

# Apply recommendation function to each row of the DataFrame
data_pagerank_train['Recommended_Items'] = data_pagerank_train.apply(lambda row: recommend_products(row['Client ID']), axis=1)

# Apply names function
data_pagerank_train['Recommendation Names'] = data_pagerank_train['Recommended_Items'].apply(map_ids_to_names)

# Display DataFrame with recommended items
data_pagerank_train.head()

,Client ID,Date,ID Product,Recommended_Items,Recommendation Names
61,448,2019-03-01,"[136, 607]","[204, 452, 260, 503, 334, 199, 273, 136, 265, ...","[Açúcar Mchef Branco Papel, Vinho Douro Planal..."
126,596,2019-03-01,"[872, 190]","[190, 204, 314, 452, 1060, 168, 158, 564, 451,...","[Vinho Frades Bco, Açúcar Mchef Branco Papel, ..."
178,648,2019-03-01,"[59, 135, 137, 541, 542, 130, 263, 236, 415, 4...","[204, 276, 407, 507, 713, 431, 452, 278, 130, ...","[Açúcar Mchef Branco Papel, Leite Mchef Uht M/..."
293,1013,2019-03-01,"[13, 71, 49, 12, 54, 90]","[204, 299, 276, 173, 142, 550, 269, 168, 364, ...","[Açúcar Mchef Branco Papel, Ref Coca Cola Orig..."
341,1275,2019-03-01,[540],"[190, 204, 139, 276, 296, 334, 152, 273, 540, ...","[Vinho Frades Bco, Açúcar Mchef Branco Papel, ..."


In [42]:
# TEST

data_pagerank_test = test_data[["Client ID","Date","ID Product"]]

# Apply recommendation function to each row of the DataFrame
data_pagerank_test['Recommended_Items'] = data_pagerank_test.apply(lambda row: recommend_products(row['Client ID']), axis=1)

# Apply names function
data_pagerank_test['Recommendation Names'] = data_pagerank_test['Recommended_Items'].apply(map_ids_to_names)

data_pagerank_test

,Client ID,Date,ID Product,Recommended_Items,Recommendation Names
10777,7971,2019-05-15,"[34, 76, 1501, 40, 95, 204, 206, 136, 273, 373...","[204, 139, 173, 734, 240, 431, 147, 239, 278, ...","[Açúcar Mchef Branco Papel, Óleo Alimentar Mch..."
10961,8021,2019-05-15,"[1310, 1590, 805, 299, 203]","[299, 296, 203, 805, 454, 1590, 1310, 1191, 31...","[Ref Coca Cola Original, Água C/gas Pedras Sal..."
11011,8025,2019-05-15,"[115, 58, 807, 376, 379, 664, 240, 540, 168, 5...","[240, 168, 189, 143, 503, 323, 379, 279, 651, ...","[Salmao Plt, Maionese Guloso, Pelicula Ader Mc..."
11084,8041,2019-05-15,"[3, 75, 204, 175, 177, 136, 328, 210, 814, 879...","[204, 407, 173, 359, 296, 227, 142, 746, 260, ...","[Açúcar Mchef Branco Papel, Licor Beirão, Det ..."
11166,8049,2019-05-15,"[2388, 87, 63, 11, 5, 14, 16, 552, 130, 231, 2...","[190, 139, 276, 407, 173, 314, 359, 550, 746, ...","[Vinho Frades Bco, Óleo Alimentar Mchef, Leite..."
...,...,...,...,...,...
17309,8872,2019-05-31,"[2431, 1103, 2383, 434, 389]","[1861, 389, 1531, 1798, 2383, 1191, 319, 133, ...","[Dourada Amanhada, Dourada Amanhada, Carvao Ve..."
17320,8876,2019-05-31,"[2416, 2405]","[190, 1453, 204, 1455, 139, 299, 276, 407, 173...","[Vinho Frades Bco, Aperitivo Martini Bianco, A..."
17333,8880,2019-05-31,"[71, 50, 51, 12, 505, 238, 1111, 143, 299]","[190, 1453, 204, 1455, 139, 299, 276, 407, 173...","[Vinho Frades Bco, Aperitivo Martini Bianco, A..."
17353,8892,2019-05-31,"[423, 250, 251, 698, 254, 377, 808, 1196, 760,...","[190, 1453, 204, 1455, 139, 299, 276, 407, 173...","[Vinho Frades Bco, Aperitivo Martini Bianco, A..."


In [43]:
# EVALUATION

train_precision, train_f1, train_hit_rate = evaluate_model_popularity_client(data_pagerank_train, 'ID Product', 'Recommended_Items')
test_precision, test_f1, test_hit_rate = evaluate_model_popularity_client(data_pagerank_test, 'ID Product', 'Recommended_Items')

print("========TRAIN========")
print("Precision: ", round(train_precision, 4), "\nF1: ", round(train_f1, 4), "\nHit Rate: ", round(train_hit_rate, 4))

print("\n========TEST========")
print("Precision: ", round(test_precision, 4), "\nF1: ", round(test_f1, 4), "\nHit Rate: ", round(test_hit_rate, 4))

========TRAIN========
Precision:  0.2691 
F1:  0.2327 
Hit Rate:  0.8383

========TEST========
Precision:  0.2411 
F1:  0.2073 
Hit Rate:  0.7979


<hr>

<a class="anchor" id="3">

# 3. Results

</a>

**Content Based Filtering**

In [44]:
product_list_cbr.head(10)

,ID Product,Product Description,Similar Recommendations,Recommendation Names
0,1,Couve Flor Mchef,"[3, 67, 1116, 1576, 55, 1385, 63, 2, 983, 45]","[Broculos Mchef Cong, Couve Bruxelas Cong Mast..."
1,2,Espinafres Folhas Cong Masterchef,"[3, 63, 204, 577, 334, 276, 12, 273, 331, 139]","[Broculos Mchef Cong, Ervilha Mchef Cong, Açúc..."
2,3,Broculos Mchef Cong,"[2, 63, 10, 1, 204, 99, 45, 12, 199, 334]","[Espinafres Folhas Cong Masterchef, Ervilha Mc..."
3,4,Grelos Amanh Cg,"[191, 190, 3, 167, 63, 800, 80, 303, 2276, 1061]","[Azeitonas Pretas S/caroço Coopoliva, Vinho Fr..."
4,5,Gel.cat.gourmes Baunilha,"[15, 16, 14, 488, 21, 560, 507, 173, 204, 175]","[Gel.cat.gourmes Morango, Gel.cat.gourmes, Gel..."
5,6,Sorvete Gourmes Tangerina,"[101, 1068, 27, 21, 2228, 1780, 15, 1823, 75, ...","[Sorvete Gourmes Manga, Gel Gourmes Leite Fr V..."
6,7,Gelo Em Cubos Mchef,"[1777, 1945, 1900, 2315, 384, 1661, 950, 2139,...","[Brigadeiro Cacau, Arroz Hakumaki Sushi Rice, ..."
7,8,Miolo De Ameijoa - Cong,"[2439, 9, 2126, 1780, 1638, 2085, 2178, 1566, ...","[Miolo Ameijoa Amanhecer, Miolo Mexilhão Cong,..."
8,9,Miolo Mexilhão Cong,"[8, 1728, 2272, 2126, 1798, 294, 1495, 2368, 1...","[Miolo De Ameijoa - Cong, Peru Peito Granel, A..."
9,10,Feijao Verde Cong. Mchefe,"[45, 3, 2, 67, 634, 577, 199, 1254, 691, 82]","[Cenoura Baby Cong. Mchefe, Broculos Mchef Con..."


**Collaborative Filtering**

In [45]:
client_list_coll.head(10)

,Client ID,Cliente Type Description,ID Product,Recommendations IDs,Recommendations Name
0,448,Hoteis 4/5 Estrelas,"[136, 607]","[276, 152, 139, 173, 299, 206, 12, 203, 168, 160]","[Leite Mchef Uht M/gordo, Farinha Mchef S/ferm..."
1,596,Catering Eventos,"[872, 190]","[276, 152, 210, 139, 173, 273, 299, 206, 203, ...","[Leite Mchef Uht M/gordo, Farinha Mchef S/ferm..."
2,648,Rodízio Carne,"[59, 135, 137, 541, 542, 130, 263, 236, 415, 4...","[136, 152, 210, 139, 173, 273, 299, 206, 12, 168]","[Vinagre Br Chef, Farinha Mchef S/fermento, Po..."
3,1013,Rodízio Carne,"[13, 71, 49, 12, 54, 90]","[152, 139, 273, 206, 203, 147, 296, 199, 443, ...","[Farinha Mchef S/fermento, Óleo Alimentar Mche..."
4,1275,Cozinha Internacional,[540],"[136, 210, 173, 299, 12, 203, 168, 160, 189, 147]","[Vinagre Br Chef, Polpa Tomate Mastercheff, De..."
5,1338,Cozinha Mexicana,"[360, 281, 158, 361, 362, 363, 162, 186, 364, ...","[204, 276, 136, 152, 210, 139, 173, 273, 299, ...","[Açúcar Mchef Branco Papel, Leite Mchef Uht M/..."
6,1394,Cozinha Portuguesa,"[66, 11, 90, 204, 177, 381, 136, 151, 1047, 15...","[299, 12, 203, 160, 407, 130, 249, 466, 205, 357]","[Ref Coca Cola Original, Frutos Vermelhos Mche..."
7,1413,Hoteis 4/5 Estrelas,"[410, 514, 374, 233, 589, 590, 414, 415]","[276, 152, 210, 139, 173, 299, 206, 203, 168, ...","[Leite Mchef Uht M/gordo, Farinha Mchef S/ferm..."
8,1528,Cozinha Internacional,"[204, 135, 136, 330, 626, 276, 466, 627, 391, ...","[210, 173, 299, 12, 203, 160, 189, 147, 296, 199]","[Polpa Tomate Mastercheff, Det Loica Mchef, Re..."
9,1892,Cozinha Autor,[1061],"[139, 273, 299, 206, 12, 203, 168, 147, 296, 199]","[Óleo Alimentar Mchef, Leite Condensado Master..."


**Popularity Based Recommendation - General**

In [46]:
for x in map_ids_to_names(top_items):
    print(x)

Açúcar Mchef Branco Papel
Leite Mchef Uht M/gordo
Óleo Alimentar Mchef
Alface Frisada Plt
Det Loica Mchef
Vinagre Br Chef
Farinha Mchef S/fermento
Ovo Agro Ovo Ind
Ovo Agro Ovo Ind
Coentros Plt


**Popularity Based Recommendation - Per Client Type**

In [47]:
pop_data_type_client = train_type_client.append(test_type_client)
pop_data_type_client[['Cliente Type Description','Recommendation Names']].drop_duplicates("Cliente Type Description").reset_index(drop=True)

,Cliente Type Description,Recommendation Names
0,Hoteis 4/5 Estrelas,"[Açúcar Mchef Branco Papel, Queijo Flamengo Vi..."
1,Catering Eventos,"[Queijo Flamengo Villas Fat, Fiambre Pa Mchef ..."
2,Rodízio Carne,"[Óleo Alimentar Mchef, Alface Frisada Plt, Açú..."
3,Cozinha Internacional,"[Açúcar Mchef Branco Papel, Óleo Alimentar Mch..."
4,Cozinha Mexicana,"[Triple Seco Olaias, Preparado Snel Para Bater..."
5,Cozinha Portuguesa,"[Açúcar Mchef Branco Papel, Leite Mchef Uht M/..."
6,Cozinha Autor,"[Açúcar Mchef Branco Papel, Frutos Vermelhos M..."
7,Bares/snacks/cafes/gelatarias/brunch,"[Leite Mchef Uht M/gordo, Queijo Flamengo Vill..."
8,Cozinha Japonesa,"[Açúcar Mchef Branco Papel, Pepino Rch, Manga ..."
9,Cozinha Italiana,"[Atum Maresia Óleo Bolsa, Ananas Guloso Pedaco..."


**Popularity Based Recommendation - Per Client**

In [48]:
pop_data_client = train_data_client.append(test_data_client)
pop_data_client.head(10)

,Date,Client ID,ID Product,Recommendations,Recommendation Names
8,2019-03-01,448,"[136, 607]","[534, 932, 352, 204, 341, 1270, 915, 136, 503,...","[Adoçante Gourmes Pó, Tomate Pedacos Mchef, Pe..."
12,2019-03-01,596,"[872, 190]","[202, 292, 1151, 1636, 177, 168, 938, 218, 947...","[Néctar Compal Clássico Laranja, Nata P/bater ..."
14,2019-03-01,648,"[59, 135, 137, 541, 542, 130, 263, 236, 415, 4...","[541, 130, 415, 304, 135, 137, 278, 236, 519, ...","[Azeitona Vde Mchef, Ovo Agro Ovo Ind, Caldo V..."
21,2019-03-01,1013,"[13, 71, 49, 12, 54, 90]","[13, 1357, 90, 160, 334, 501, 399, 323, 643, 339]","[Bat. Pal. Mchef Cong, Porco Bifanas, Frango P..."
23,2019-03-01,1275,[540],"[540, 135, 1049, 276, 628, 206, 334, 213, 204,...","[Dourada Rch, Arroz Cozinha Velha Agulha, Cald..."
25,2019-03-01,1338,"[360, 281, 158, 361, 362, 363, 162, 186, 364, ...","[362, 158, 162, 186, 364, 360, 365, 361, 949, ...","[Feijão Preto Mchef, Óleo Alimentar Mchef, Tom..."
27,2019-03-01,1394,"[66, 11, 90, 204, 177, 381, 136, 151, 1047, 15...","[139, 177, 470, 158, 601, 223, 90, 151, 753, 381]","[Óleo Alimentar Mchef, Arroz Agulha Masterchef..."
28,2019-03-01,1413,"[410, 514, 374, 233, 589, 590, 414, 415]","[379, 233, 412, 414, 589, 238, 504, 514, 415, ...","[Alface Frisada Plt, Morango Rch, Manga Avião ..."
29,2019-03-01,1528,"[204, 135, 136, 330, 626, 276, 466, 627, 391, ...","[135, 391, 139, 628, 204, 168, 627, 206, 334, ...","[Arroz Cozinha Velha Agulha, Caldo Galinha Kno..."
36,2019-03-01,1892,[1061],"[537, 484, 354, 156, 491, 152, 500, 308, 331, ...","[Açúcar Po Icing Henrinor, Espte.leao Dourado ..."


**"Did You Forget?"**

In [49]:
forget_rec_data.head(10)

,Client ID,Date,ID Client Type,Cliente Type Description,ID Product,Recommendations,Recommendation Names,Basket Names
10777,7971,2019-05-15,201,Cozinha Portuguesa,"[34, 76, 1501, 40, 95, 204, 206, 136, 273, 373...","[157, 278, 1395, 812, 1483, 324, 1468, 1089, 4...","[Leite Est.atlantico Uht M/g, Manteiga Mimosa ...","[Bacalhau Lombo Ps Iqf Cg, Miolo Camarao Cg, C..."
10961,8021,2019-05-15,214,Cozinha Japonesa,"[1310, 1590, 805, 299, 203]","[296, 454]","[Água C/gas Pedras Salgadas, Massa Mchef Fusil...","[Camarão Bt Cg, Arroz Carolino Masterchef, Ref..."
11011,8025,2019-05-15,201,Cozinha Portuguesa,"[115, 58, 807, 376, 379, 664, 240, 540, 168, 5...","[663, 38, 377, 1346, 887, 2293, 345, 1645, 737...","[Batata Conservação Assar Verm Rch, Camarao S/...","[Pastel De Nata Mchef, Frango Peito Cong Saco,..."
11084,8041,2019-05-15,203,Cozinha Chinesa,"[3, 75, 204, 175, 177, 136, 328, 210, 814, 879...","[878, 282, 320, 353, 427, 2047, 206, 330, 2290...","[Bovino Maminha, Bolacha Amanhecer Maria, Vaqu...","[Broculos Mchef Cong, Porco Lombo S/o Cong C/ ..."
11166,8049,2019-05-15,214,Cozinha Japonesa,"[2388, 87, 63, 11, 5, 14, 16, 552, 130, 231, 2...","[270, 345, 15, 232, 233, 351, 331, 173, 27, 350]","[Bacon Fatiado Masterchef, Cebola Rch, Gel.cat...","[Bat. Pal. Ecofrost Cong, Sob. Tranche Bolacha..."
11256,8056,2019-05-15,214,Cozinha Japonesa,"[234, 235]","[586, 236, 232, 251, 233, 250, 699, 231, 76, 58]","[Salmao Rch, Rucúla Selvagem Hotform Rch, Pepi...","[Cenoura Ripada Rch, Alho Francês Cortado Mche..."
11338,8097,2019-05-15,214,Cozinha Japonesa,"[204, 466, 970, 260, 859, 503]","[152, 97, 207, 168, 26, 371, 176, 135, 241, 161]","[Farinha Mchef S/fermento, Delícias Do Mar S/g...","[Açúcar Mchef Branco Papel, Sal Mchef Mesa, Ól..."
11397,8099,2019-05-15,201,Cozinha Portuguesa,"[23, 12, 814, 290, 246, 577, 988]","[932, 431, 424, 719, 568, 293, 557, 245, 1895,...","[Tomate Pedacos Mchef, Natas Gresso Chef Uht, ...","[Espargos Verdes Bonduelle Cong, Frutos Vermel..."
11470,8100,2019-05-15,201,Cozinha Portuguesa,"[41, 826, 2276, 178, 249, 424, 270, 431, 841, ...","[17, 246, 422, 1436, 315, 530, 492, 1095, 569,...","[Salmao Fumado Ubago Precort, Queijo Ilha Cunh...","[Pasteis Bacalhau Mchef Cg, Massa Nacional Mac..."
11542,8101,2019-05-15,202,Cozinha Italiana,"[17, 23, 754, 431, 557, 466, 160, 440, 568, 56...","[293, 578, 139, 803, 130, 492, 422, 152, 204, 12]","[Manteiga Gresso C/sal, Miolo Noz Mchef Metade...","[Salmao Fumado Ubago Precort, Espargos Verdes ..."


**Smart Basket Analysis**

In [50]:
recom_smart_basket.head(10)

,Client ID,Date,ID Client Type,Cliente Type Description,ID Product,Recommended_Items,Recommendation Names,Basket Names
61,448,2019-03-01,210,Hoteis 4/5 Estrelas,"[136, 607]","[139, 152, 168, 173, 175, 177, 204, 206, 210, ...","[Óleo Alimentar Mchef, Farinha Mchef S/ferment...","[Vinagre Br Chef, Bebida Soja Vive Soy Natural]"
126,596,2019-03-01,226,Catering Eventos,"[872, 190]",[],[],"[Cogumelos Inteiros Mast.chef, Vinho Frades Bco]"
178,648,2019-03-01,219,Rodízio Carne,"[59, 135, 137, 541, 542, 130, 263, 236, 415, 4...","[3, 12, 139, 173, 177, 204, 210, 237, 276, 130...","[Broculos Mchef Cong, Frutos Vermelhos Mchef, ...","[Salada Russa Cong Mchef, Arroz Cozinha Velha ..."
293,1013,2019-03-01,219,Rodízio Carne,"[13, 71, 49, 12, 54, 90]","[3, 139, 161, 173, 204, 276, 379]","[Broculos Mchef Cong, Óleo Alimentar Mchef, At...","[Bat. Pal. Mchef Cong, Polpa Fruta Cong Morang..."
341,1275,2019-03-01,211,Cozinha Internacional,[540],[],[],[Dourada Rch]
373,1338,2019-03-01,213,Cozinha Mexicana,"[360, 281, 158, 361, 362, 363, 162, 186, 364, ...","[139, 204, 276]","[Óleo Alimentar Mchef, Açúcar Mchef Branco Pap...","[Arroz Caçarola Estufado, Ananas Guloso Rod, Ó..."
404,1394,2019-03-01,201,Cozinha Portuguesa,"[66, 11, 90, 204, 177, 381, 136, 151, 1047, 15...","[3, 5, 12, 13, 26, 63, 130, 139, 136, 152, 168...","[Broculos Mchef Cong, Gel.cat.gourmes Baunilha...","[Massa Folhada Solara Placas Rch, Milho Doce C..."
441,1413,2019-03-01,210,Hoteis 4/5 Estrelas,"[410, 514, 374, 233, 589, 590, 414, 415]","[232, 236, 250, 699, 379]","[Pepino Rch, Rucúla Selvagem Hotform Rch, Mang...","[Maçã Reineta Rch, Limas Rch, Cogumelo Branco ..."
472,1528,2019-03-01,211,Cozinha Internacional,"[204, 135, 136, 330, 626, 276, 466, 627, 391, ...","[3, 5, 12, 13, 26, 63, 130, 139, 152, 168, 173...","[Broculos Mchef Cong, Gel.cat.gourmes Baunilha...","[Açúcar Mchef Branco Papel, Arroz Cozinha Velh..."
606,1892,2019-03-01,208,Cozinha Autor,[1061],[],[],[Pau Canela Ind Condi]


**Page Rank**

In [51]:
data_pagerank = data_pagerank_train.append(data_pagerank_test)
data_pagerank.head(10)

,Client ID,Date,ID Product,Recommended_Items,Recommendation Names
61,448,2019-03-01,"[136, 607]","[204, 452, 260, 503, 334, 199, 273, 136, 265, ...","[Açúcar Mchef Branco Papel, Vinho Douro Planal..."
126,596,2019-03-01,"[872, 190]","[190, 204, 314, 452, 1060, 168, 158, 564, 451,...","[Vinho Frades Bco, Açúcar Mchef Branco Papel, ..."
178,648,2019-03-01,"[59, 135, 137, 541, 542, 130, 263, 236, 415, 4...","[204, 276, 407, 507, 713, 431, 452, 278, 130, ...","[Açúcar Mchef Branco Papel, Leite Mchef Uht M/..."
293,1013,2019-03-01,"[13, 71, 49, 12, 54, 90]","[204, 299, 276, 173, 142, 550, 269, 168, 364, ...","[Açúcar Mchef Branco Papel, Ref Coca Cola Orig..."
341,1275,2019-03-01,[540],"[190, 204, 139, 276, 296, 334, 152, 273, 540, ...","[Vinho Frades Bco, Açúcar Mchef Branco Papel, ..."
373,1338,2019-03-01,"[360, 281, 158, 361, 362, 363, 162, 186, 364, ...","[365, 364, 158, 186, 391, 381, 162, 362, 361, ...","[Espumante Asso Moscato Malvasia, Triple Seco ..."
404,1394,2019-03-01,"[66, 11, 90, 204, 177, 381, 136, 151, 1047, 15...","[190, 204, 139, 276, 173, 314, 296, 147, 168, ...","[Vinho Frades Bco, Açúcar Mchef Branco Papel, ..."
441,1413,2019-03-01,"[410, 514, 374, 233, 589, 590, 414, 415]","[204, 199, 379, 259, 273, 491, 177, 539, 136, ...","[Açúcar Mchef Branco Papel, Pickles Guloso, Al..."
472,1528,2019-03-01,"[204, 135, 136, 330, 626, 276, 466, 627, 391, ...","[190, 204, 139, 276, 168, 334, 141, 547, 152, ...","[Vinho Frades Bco, Açúcar Mchef Branco Papel, ..."
606,1892,2019-03-01,[1061],"[190, 204, 276, 173, 485, 189, 354, 160, 152, ...","[Vinho Frades Bco, Açúcar Mchef Branco Papel, ..."
